# Setting

라이브러리 다운로드

In [ ]:
# !apt-get install g++
# !pip install pororo 

In [ ]:
# https://github.com/SKTBrain/KoBERT
# 여기 koBert랑 KoGPT2가 있는데 같이 사용해서 앙상블하면,,,좋지않을까,,,
# 아니면 다국어 모델?도 좋을거같은데 
# ERNIE, ELECTRA, GPT-2 이런거...? ㅎㅎ,,,,
# KoGPT2에 토크나이저만 바꾸는건 에반가

라이브러리 불러오기

In [15]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
import re
import tarfile
import pickle as pickle
from tqdm import tqdm
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup, LambdaLR
from sklearn.model_selection import train_test_split
import random
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
import os
from glob import glob
from tokenizers import BertWordPieceTokenizer
from pororo import Pororo
from collections import Counter

Error in callback <function _WandbInit._resume_backend at 0x7f7080fea950> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7f7080fea8c0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [5]:
wandb.login()
wandb.init(project = 'Pstage2_KLUE', reinit = True)
wandb.run.name = 'koBert'

wandb: Currently logged in as: ahnyujin (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [2]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed = 7
seed_everything(seed)

GPU 설정

In [3]:
device = torch.device("cuda:0")

kobert 불러오기

In [4]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


# Preprocessing

In [6]:
def load_data(dataset):
    with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
        label_type = pickle.load(f)
#    dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
    dataset = preprocessing_dataset(dataset, label_type)
    return dataset

def preprocessing_dataset(dataset, label_type):
    label = []
    for i in dataset[8]:
        if i == 'blind':
            label.append(100)
        else:
            label.append(label_type[i])
    out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
    return out_dataset

In [7]:
def add_entity_tokens(sentence, a1, a2, b1, b2):
    new_sentence = None
    ner = Pororo(task="ner", lang="ko")
    
    if a1 > b1:
        e1, e2 = sentence[b1:b2+1], sentence[a1:a2+1]
        n1, n2 = Counter([e[1] for e in ner(e1)]).most_common(1)[0][0], Counter([e[1] for e in ner(e2)]).most_common(1)[0][0]
        ner1, ner2 = "*" + n1 + "*", "^" + n2 + "^"
        new_sentence = sentence[:b1] + "@" + ner1 + sentence[b1:b2+1] + "@" + sentence[b2+1:a1] + "#" + ner2 + sentence[a1:a2+1] + "#" + sentence[a2+1:]
    else:
        e1, e2 = sentence[a1:a2+1], sentence[b1:b2+1]
        n1, n2 = Counter([e[1] for e in ner(e1)]).most_common(1)[0][0], Counter([e[1] for e in ner(e2)]).most_common(1)[0][0]
        ner1, ner2 = "^" + n1 + "^", "*" + n2 + "*"
        new_sentence = sentence[:a1] + "#" + ner1 + sentence[a1:a2+1] + "#" + sentence[a2+1:b1] + "@" + ner2 + sentence[b1:b2+1] + "@" + sentence[b2+1:]
    return new_sentence

In [ ]:
dataset_path = r"/opt/ml/input/data/train/train.tsv"
dataset = pd.read_csv(dataset_path, delimiter='\t', header=None)

In [ ]:
# dataset_path = r"/opt/ml/input/data/train/train.tsv"
# dataset = pd.read_csv(dataset_path, delimiter='\t', header=None)
# d = []
# for idx,i in enumerate(dataset[1]):
#     # d.append(add_entity_tokens(i, dataset[3][idx], dataset[4][idx], dataset[6][idx], dataset[7][idx]))
#     dataset[1][idx] = add_entity_tokens(i, dataset[3][idx], dataset[4][idx], dataset[6][idx], dataset[7][idx])

In [14]:
# dataset.to_csv("/opt/ml/input/data/train/train_with_label.txt", sep='\t',index=False) 

Error in callback <function _WandbInit._resume_backend at 0x7f7080fea950> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7f7080fea8c0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
dataset['sentence'] = add_entity_tokens(dataset['sentence'], data)

In [ ]:
dataset['sentence'] = dataset['entity_01'] + ' [SEP] ' + dataset['entity_02'] + ' [SEP] ' + dataset['sentence']
# dataset['sentence'] = dataset['sentence'] +  '이 문장에서 ' + dataset['entity_01'] + '과 ' + dataset['entity_02'] + "은 어떤 관계일까?[SEP]"
                                                                                              

In [ ]:
# dataset[['sentence', 'label']].to_csv("/opt/ml/input/data/train/train.txt", sep='\t', index=False)
train, vali = train_test_split(dataset, test_size=0.1, random_state=42)
train[['sentence','label']].to_csv("/opt/ml/input/data/train/train_train.txt", sep='\t',index=False) # sep='\t',
vali[['sentence','label']].to_csv("/opt/ml/input/data/train/train_vali.txt",  sep='\t',index=False)

In [ ]:
# dataset = nlp.data.TSVDataset("/opt/ml/input/data/train/train.txt", field_indices=[0,1], num_discard_samples=1, allow_missing = True)
dataset_train = nlp.data.TSVDataset("/opt/ml/input/data/train/train_train.txt", field_indices=[0,1], num_discard_samples=1, allow_missing = True)
dataset_vali = nlp.data.TSVDataset("/opt/ml/input/data/train/train_vali.txt", field_indices=[0,1], num_discard_samples=1,allow_missing = True)

## Tokenizer

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer,vocab = vocab, lower=False)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 128
batch_size = 32
warmup_ratio = 0.01
num_epochs = 20
max_grad_norm = 1
log_interval = 50
learning_rate = 5e-5

wandb.config.update({
    "model" : "koBERT",
    "loss" : "focal_loss",
    "max_len" : 128,
    "batch_size" : 32,
    "warmup_ratio" : 0.01,
    "num_epochs" : 20,
    "max_grad_norm" : 1,
    "log_interval" : 50,
    "learning_rate" : 5e-5,
    "dr_rate" : 0.5
})

In [ ]:
#data = BERTDataset(dataset, 0, 1,tok, max_len, True, False)
data_train = BERTDataset(dataset_train, 0, 1,tok, max_len, True, False)
data_vali = BERTDataset(dataset_vali, 0, 1,tok, max_len, True, False)
# print(data_train[0])
# [0][0] : token_ids : Token Embedding을 생성하는데 필요
# [0][1] : valid_len : Positional Embedding을 위한것
# [0][2] : token_types : Sentence Embedding
# token_ids, valid_length, segment_ids, label

In [ ]:
#train_dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=5)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
vali_dataloader = torch.utils.data.DataLoader(data_vali, batch_size=batch_size, num_workers=5)

# Classification

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 42,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
wandb.watch(model)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=42, smoothing=0.2, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    

class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )


In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = FocalLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    best_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(vali_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    if test_acc >= best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "/opt/ml/model/model_state_dict.pt")

In [ ]:
# [32,128]

# Predict

In [ ]:
dataset_path = r"/opt/ml/input/data/test/test.tsv"

dataset = load_data(dataset_path)

dataset['sentence'] = dataset['entity_01'] + ' [SEP] ' + dataset['entity_02'] + ' [SEP] ' + dataset['sentence']
# dataset['sentence'] = dataset['sentence'] +  '이 문장에서 ' + dataset['entity_01'] + '과 ' + dataset['entity_02'] + "은 어떤 관계일까?[SEP]"
dataset[['sentence','label']].to_csv("/opt/ml/input/data/test/test.txt", sep='\t', index=False)

In [ ]:
print(dataset['sentence'][0])

In [ ]:
dataset_test = nlp.data.TSVDataset("/opt/ml/input/data/test/test.txt", field_indices=[0,1], num_discard_samples=1)

data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=10, num_workers=5)

In [ ]:
model.load_state_dict(torch.load("/opt/ml/model/model_state_dict.pt"))

model.eval()

Predict = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(out,1)
    Predict.extend(predict.tolist())

In [ ]:
output = pd.DataFrame(Predict, columns=['pred'])
output.to_csv('/opt/ml/result/submission.csv', index=False)

In [ ]:
del model
torch.cuda.empty_cache()

In [ ]:
torch.cuda.is_available()